<a href="https://colab.research.google.com/github/zoltan23/Audio-Classification/blob/master/audio_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python_speech_features
!pip install scipy
!pip install pandas
!pip install keras
!pip install sklearn
!pip install tqdm
!pip install librosa
!apt-get install libsndfile1 -y
!apt-get install git -y

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=2eb16b2c1c605eb5d042225389f5872189c624171142476b96cdfa5c65822816
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
os.chdir('/content/gdrive/My Drive/music-wav-files')

clean  instruments.csv	model  models  samples	wavfiles


In [ ]:
!mkdir clean samples model

mkdir: cannot create directory ‘clean’: File exists
mkdir: cannot create directory ‘samples’: File exists
mkdir: cannot create directory ‘model’: File exists


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from scipy.io import wavfile as wav
import numpy as np

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics

# The following code was extracted from a module that was created to train a CNN model.  It will be called to retrain the model to improve accuracy as more data becomes available from the app.  

In [ ]:
def extract_features(file_name):

    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

        mask = envelope(audio, sample_rate, 0.0005)

        mfccs = librosa.feature.mfcc(y=audio[mask], sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)

        #print("mfccsscaled", mfccsscaled.shape)
    
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
    
    return mfccsscaled

def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10), min_periods=1, center=True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

fulldatasetpath = '/wavfiles/'

metadata = pd.read_csv('instruments.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():

    file_name = os.path.join(os.path.abspath(fulldatasetpath), str(row["fname"]))

    class_label = row["label"]
    data = extract_features(file_name)
    features.append([data, class_label])

    # Convert into a Panda dataframe 
    featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

    #print('Finished feature extraction from ', len(featuresdf), ' files') 


# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

num_labels = yy.shape[1]
print("num_labels", num_labels)
filter_size = 2

model = Sequential()
model.add(Dense(256, input_shape=(40,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_labels, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

num_epochs = 15
num_batch_size = 8

start = datetime.now()

#callbacks=[checkpointer],
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test),  verbose=1)
model.save('/models/model4.h5')

duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Error encountered while parsing file:  /wavfiles/5388d14d.wav
Error encountered while parsing file:  /wavfiles/c685f05f.wav
Error encountered while parsing file:  /wavfiles/36d20ab5.wav
Error encountered while parsing file:  /wavfiles/d6665734.wav
Error encountered while parsing file:  /wavfiles/7352e28f.wav
Error encountered while parsing file:  /wavfiles/8fa096bf.wav
Error encountered while parsing file:  /wavfiles/2f9c5f62.wav
Error encountered while parsing file:  /wavfiles/9cd05a2b.wav
Error encountered while parsing file:  /wavfiles/ab465bf9.wav
Error encountered while parsing file:  /wavfiles/c40632d7.wav
Error encountered while parsing file:  /wavfiles/3a80061f.wav
Error encountered while parsing file:  /wavfiles/dba6817d.wav
Error encountered while parsing file:  /wavfiles/25f26095.wav
Error encountered while parsing file:  /wavfiles/37cebbd4.wav
Error encountered while parsing file:  /wavfiles/f8d96888.wav
Error encountered while parsing file:  /wavfiles/d94c94f0.wav
Error en

ValueError: ignored

In [ ]:
def classify(filename):
    #model = load_model('/tf/desktop/music-wav-files/models/model1.h5')
    model = load_model('/tf/desktop/music-wav-files/models/model3.h5')

    def extract_feature(file_name):
   
        try:
            audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
            mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
            mfccsscaled = np.mean(mfccs.T,axis=0)
            
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)
            return None, None
        print("np.array", np.array([mfccsscaled]))
        return np.array([mfccsscaled])

    #Get class labels for predicted_instrument output
    le = LabelEncoder()
    metadata = pd.read_csv('/tf/desktop/music-wav-files/instruments.csv')
    le.fit(metadata['label'])

    def print_prediction(file_name):
  
        prediction_feature = extract_feature(file_name) 

        predicted_vector = model.predict_classes(prediction_feature)
        #print("predicted_vector", predicted_vector)

        predicted_class = le.inverse_transform(predicted_vector)
        predicted_instrument = predicted_class[0]
        print("The predicted class is:", predicted_class[0], '\n') 

        predicted_proba_vector = model.predict_proba(prediction_feature) 
        predicted_proba = predicted_proba_vector[0]
        for i in range(len(predicted_proba)):
            category = le.inverse_transform(np.array([i]))
            print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )
            
        return predicted_instrument

    
    return print_prediction(filename)
    


# The following code was extracted from the unit tests we created.

In [ ]:
def predictedInstrumentTest(filename, instrument):

    predicted_instrument = classify(filename)
    print("The predicted instrument is", predicted_instrument, ". The actual instrument is", instrument, ".")
    if (instrument == predicted_instrument):
        print("Test passed!!!")
    else:
        print("Test Failed")

In [ ]:
predictedInstrumentTest('/tf/desktop/cl_C4.wav', 'Clarinet')
predictedInstrumentTest('/tf/desktop/BbClarinet.wav', 'Clarinet')
predictedInstrumentTest('/tf/desktop/trumpet4.wav', 'Trumpet')
predictedInstrumentTest('/tf/desktop/trumpet5.wav', 'Trumpet')
predictedInstrumentTest('/tf/desktop/g_sharp_real_mono.wav', 'Trumpet')
predictedInstrumentTest('/tf/desktop/trumpet_real2.wav', 'Trumpet')